In [1]:
import cv2
import numpy as np

import tensorflow as tf
import keras
from keras import applications
from keras.models import Model, load_model
from keras.utils import np_utils

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, roc_auc_score

Using TensorFlow backend.


In [2]:
from google.colab import drive

def load_drive():
  drive.mount('/content/drive')
  base_dir = '/content/drive/My Drive/COVID/16CO154-16CV245-COVID/'
  return base_dir

# Load data
base_dir = load_drive()
x_test = np.load(base_dir + 'Test_Data/testx.npy')
y_test = np.load(base_dir + 'Test_Data/testy.npy')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def reshape(c):
  reshaped = np.empty((len(c), 256, 256, 3), dtype = np.uint8)
  dim = (256, 256)

  for i in range(len(c)):
    assert(c[i].shape == (512, 512, 3))
    reshaped[i] = cv2.resize(c[i], dim, interpolation = cv2.INTER_CUBIC)
  
  del c
  return reshaped

In [4]:
model = load_model('/content/drive/My Drive/COVID/16CO154-16CV245-COVID/Model/separablenet.hdf5')

xtest = reshape(x_test)
xtest = (xtest.astype('float32')) / 255.0

y_pred = np.argmax(model.predict(xtest), axis=1)

target_names = ['COVID-19', 'Normal', 'Viral Pneumonia']
print(classification_report(y_test, y_pred, target_names=target_names, digits=6))
print('\n Confusion Matrix: ')
print(confusion_matrix(y_test, y_pred))

                 precision    recall  f1-score   support

       COVID-19   1.000000  1.000000  1.000000        49
         Normal   0.981061  0.962825  0.971857       269
Viral Pneumonia   0.963504  0.981413  0.972376       269

       accuracy                       0.974446       587
      macro avg   0.981521  0.981413  0.981411       587
   weighted avg   0.974596  0.974446  0.974444       587


 Confusion Matrix: 
[[ 49   0   0]
 [  0 259  10]
 [  0   5 264]]


In [5]:
tn_cmat = np.multiply(y_test!=0, y_test==y_pred)
tn_c = np.count_nonzero(tn_cmat)
fp_cmat = np.multiply(y_test!=0, y_pred==0)
fp_c = np.count_nonzero(fp_cmat)

sp_c = tn_c/(tn_c+fp_c)
print("Covid: ", sp_c)

tn_nmat = np.multiply(y_test!=1, y_test==y_pred)
tn_n = np.count_nonzero(tn_cmat)
fp_nmat = np.multiply(y_test!=1, y_pred==1)
fp_n = np.count_nonzero(fp_cmat)

sp_n = tn_n/(tn_n+fp_n)
print("Normal: ", sp_n)

tn_vmat = np.multiply(y_test!=2, y_test==y_pred)
tn_v = np.count_nonzero(tn_vmat)
fp_vmat = np.multiply(y_test!=2, y_pred==2)
fp_v = np.count_nonzero(fp_vmat)

sp_v = tn_v/(tn_v+fp_v)
print("Viral: ", sp_v)

sp_total = sp_c *np.count_nonzero(y_test==0) + sp_n *np.count_nonzero(y_test==1) + sp_v *np.count_nonzero(y_test==2)
sp = sp_total/y_test.shape[0]
print("Specificity: ", sp)

Covid:  1.0
Normal:  1.0
Viral:  0.9685534591194969
Specificity:  0.9855892342472652
